In [71]:
import torch
from numpy import asarray,stack,concatenate,convolve,ones,arange,stack,log,sqrt
from glob import glob

from matplotlib.pyplot import plot,semilogx,semilogy,subplots,legend,fill_between,xlim,ylim

import re

In [103]:
def read_loss(path):
    checkpoints = sorted(glob(path + '/*.pth'))
    loss = []
    test_loss = []
    

    for checkpoint in checkpoints:
        loss += [torch.load(checkpoint,map_location=torch.device('cpu'))['loss']]
        test_loss += [torch.load(checkpoint,map_location=torch.device('cpu'))['test_loss']]
    
    return asarray(loss),asarray(test_loss)

def loss_curves(path):

    folders = glob(path + '*/')
    folders = sorted(folders,key = lambda x: int(re.findall(r'\d+',x)[-1]))
    print(folders)

    loss = []
    test_loss = []
    
    for folder in folders[:]:
        
        l,v = read_loss(folder)

        loss += [l.reshape(-1,5)]
        test_loss += [v.mean(1)]
        
    return loss,test_loss

In [104]:
loss,test_loss = loss_curves('../run/ResNet1_0_2')
loss_nowarm,test_loss_nowarm = loss_curves('../run/ResNet1_0_3')

['../run\\ResNet1_0_2\\']
['../run\\ResNet1_1_1\\']


In [107]:
%matplotlib qt
fig,ax = subplots(5,1,figsize=(8,21))

for i,training in enumerate(loss):
    for j,element in enumerate(training.T):
        ax[j].set_title('train loss element '+ str(j))
        ax[j].plot(element,alpha=0.33, label='warmed')
        ax[j].plot(loss_nowarm[i].T[j],alpha=0.33, label='nowarm')
        ax[j].legend(frameon=True)
        ax[j].grid(True)

In [96]:
fig,ax = subplots(5,1,figsize=(8,21))

for i,training in enumerate(test_loss):
    for j,element in enumerate(training.T):
        ax[j].plot(element,'.-',alpha=0.33)

In [84]:
model2 = torch.load('../run/N_ElementCNN_1_0_1/0001.pth', map_location=torch.device('cpu'))

weights_dict = {
    'conv_0.weight': [torch.tensor(-0.0412), torch.tensor(0.3478)],
    'conv_1.weight': [torch.tensor(-0.0035), torch.tensor(0.1136)],
    'conv_2.weight': [torch.tensor(-0.0029), torch.tensor(0.0832)],
    'conv_3.weight': [torch.tensor(-0.0041), torch.tensor(0.0618)]
}

for i, checkp in enumerate(glob('../run/N_ElementCNN_1_0_1/0010.pth')):
    model = torch.load(checkp, map_location=torch.device('cpu'))

    for k, weights in model['model_state_dict'].items():
        if 'conv' in k and 'weight' in k:
            mean_ = weights_dict[k][0]
            std_ =  weights_dict[k][1]
            print(k, mean_, std_)
            torch.nn.init.normal_(model2['model_state_dict'][k], mean=mean_, std=std_)

conv_0.weight
conv_0.bias
conv_1.weight
conv_1.bias
conv_2.weight
conv_2.bias
conv_3.weight
conv_3.bias
fc0.weight
fc0.bias
fc1.weight
fc1.bias
